# 모델 학습


In [ ]:
# 학습을 위해서 train, val 섞어서 나누기
%cp train.txt train_full.txt
%cp val.txt   val_full.txt

# 무작위 섞기
!shuf train_full.txt > train_shuf.txt
!shuf val_full.txt   > val_shuf.txt

# train을 2,000줄씩 샤딩 (train_shard_00.txt, train_shard_01.txt, ...)

!split -l 2000 -d -a 2 train_shuf.txt train_shard_
!for f in train_shard_*; do mv "$f" "${f}.txt"; done

# val은 500줄
!head -n 500 val_shuf.txt > val_small.txt

%cd /content/drive/MyDrive/tacotron2

from glob import glob

shards = sorted(glob("/content/drive/MyDrive/tacotron2/filelists/train_shard_*.txt"))
for i,f in enumerate(shards):
    print(f">>> training on {f}")

    !python /content/drive/MyDrive/tacotron2/train.py \
      --output_directory /content/drive/MyDrive/tacotron2/output/korean-jamo \
      --log_directory /content/drive/MyDrive/tacotron2/logs \
      --checkpoint_path /content/drive/MyDrive/tacotron2/tacotron2_statedict_gpu.pt \
      --warm_start \
      --hparams "training_files=$f,validation_files=/content/drive/MyDrive/tacotron2/filelists/val_small.txt,epochs=300,iters_per_checkpoint=2000"
# !python /content/drive/MyDrive/tacotron2/train.py \
#   --output_directory /content/drive/MyDrive/tacotron2/output/korean-roman \
#   --log_directory /content/drive/MyDrive/tacotron2/logs \
#   --checkpoint_path /content/drive/MyDrive/tacotron2/tacotron2_statedict_gpu.pt \
#   --warm_start \
#   --hparams "training_files=/content/drive/MyDrive/tacotron2/filelists/train.txt,validation_files=/content/drive/MyDrive/tacotron2/filelists/val.txt,epochs=10000"

In [ ]:
import librosa
import soundfile as sf
import os
from glob import glob

# 훈련 모델에서 사용하는 음성 주파수와 데이터셋의 주파수를 맞추는 전처리
# 22050 으로 변경하는 로직

TARGET_DIR = 'kss/'  # KSS 데이터셋의 .wav 파일이 있는 경로
TARGET_SR = 22050
ORIGINAL_SR = 44100
# -----------------

# 모든 wav 파일 경로 찾기 (재귀적으로 검색)
wav_files = glob(os.path.join(TARGET_DIR, '**', '*.wav'), recursive=True)

print(f"총 {len(wav_files)}개의 파일을 22050 Hz로 변환하여 덮어씌웁니다.")
print("원본 파일 백업 여부를 확인하세요!")

for i, file_path in enumerate(wav_files):
    try:
        # 1. 오디오 파일 로드
        y, sr = librosa.load(file_path, sr=ORIGINAL_SR)

        # 2. 리샘플링 수행 (44100 Hz -> 22050 Hz)
        y_resampled = librosa.resample(y, orig_sr=ORIGINAL_SR, target_sr=TARGET_SR)

        # 3. 리샘플링된 파일 저장 (원본 파일 덮어쓰기)
        # sf.write는 기존 파일을 삭제하고 새 파일로 대체합니다.
        sf.write(file_path, y_resampled, TARGET_SR, format='WAV', subtype='PCM_16')

        if (i + 1) % 100 == 0:
            print(f"--- {i + 1}개 파일 변환 완료: {file_path} ---")

    except Exception as e:
        print(f"오류 발생: {file_path}. 오류 내용: {e}")
        continue

print("모든 파일 변환 및 덮어쓰기가 완료되었습니다.")

# 학습한 모델을 가지고 추론을 할 경우

In [ ]:
import matplotlib.pylab as plt

def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    if len(data) == 1:
        axes = [axes]  # 단일 축 대응
    for i in range(len(data)):
        axes[i].imshow(
            data[i],
            aspect='auto',
            origin='lower',    # ← 'bottom' 대신 'lower'
            interpolation='none'
        )
    plt.tight_layout()
    return fig

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

waveglow = torch.hub.load(
    'NVIDIA/DeepLearningExamples:torchhub',
    'nvidia_waveglow',
    model_math='fp32',     # 또는 'fp16' (fp16 쓰면 mel/모델도 half로 맞추세요)
    pretrained=True,
    trust_repo=True,
    force_reload=False,
).to(device).eval()

In [ ]:
%cd /content/drive/MyDrive/tacotron2

import torch
from train import create_hparams
from train import load_model
from text import text_to_sequence

def test(text, file, num):
  hparams = create_hparams("distributed_run=False")

  tacotron2 = load_model(hparams).to("cuda:0").eval()
  ckpt_path = file  # 실제 파일명
  try:
    ckpt = torch.load(ckpt_path, map_location="cuda:0", weights_only=True)
  except Exception:
      ckpt = torch.load(ckpt_path, map_location="cuda:0", weights_only=False)

  sd = ckpt.get('state_dict',ckpt)
  tacotron2.load_state_dict(sd,strict=False)

  seq = torch.LongTensor(
      text_to_sequence(text, hparams.text_cleaners)
  ).unsqueeze(0).to("cuda:0")
  import time

  start = time.perf_counter()
  with torch.no_grad():
    mel_outputs, mel_postnet, _, alignments = tacotron2.inference(seq)
  end = time.perf_counter()
  elapsed = end - start

  print(str(elapsed) + "초 걸림")

  mel = mel_postnet.squeeze(0).cpu()
  mel = mel_postnet.squeeze(0).cpu()

  fig = plot_data((mel_outputs.float().data.cpu().numpy()[0],
          mel_postnet.float().data.cpu().numpy()[0],
          alignments.float().data.cpu().numpy()[0].T))

  fig.savefig("/content/" + file.split("/")[-1] + "-" + num + "-out.png")
  plt.close(fig)

  with torch.no_grad():
    audio = waveglow.infer(mel_outputs, sigma=0.7)          # audio: [1, n_samples]
    audio = denoiser(audio, strength=0.01)[:, 0, :] # 옵션: 약한 잔향 제거
    audio = audio.squeeze(0).float().cpu().numpy()

  # 3) 저장
  import soundfile as sf
  sf.write( "/content/" + file.split("/")[-1] + "-" + num +  '.wav', audio, samplerate=22050)

# WER/CER 로 에러 비교


In [ ]:
import whisper
from jiwer import wer
from jiwer import wer, transforms as tr
from jiwer import cer


def t(audio_path):
  transform = tr.Compose([
    tr.Strip(),                # 앞뒤 공백 제거
    tr.ToLowerCase(),          # 소문자
    tr.RemoveMultipleSpaces(), # 여러 공백 -> 하나
  ])

# 1) Whisper 모델 로드 (용량 따라 tiny/small/medium 등 선택)
  model = whisper.load_model("small")  # "base", "small", "medium" 등 가능
  ref_text = "간장 공장 공장장은 강 공장장이고 된장 공장 공장장은 장 공장장이다."

  # 3) 음성 → 텍스트 (한국어라면 language="ko" 권장)
  result = model.transcribe(audio_path, language="ko")
  hyp_text = result["text"].strip()
  ref_norm = transform(ref_text)
  hyp_norm = transform(hyp_text)
  # 각 샘플 WER
  sample_wer = wer(ref_norm, hyp_norm)
  sample_cer = cer(ref_norm, hyp_norm)
  print(f"[{audio_path}]")
  print(f"  원본: {ref_text}")
  print(f"  음성: {hyp_text}")
  print(f"  WER: {sample_wer:.3f}\n")
  print(f"  CER: {sample_cer:.3f}\n")